In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_SYNC_ACCELERATED_UPDATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### MITb Dataset

In [2]:
TRIES = 10

for it in range(TRIES):

    file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_INTEL_g2o.g2o'
    data_gen.generate_SLAM_dataset_PSD(file_path=file_path)

    dataset = 'input_INTEL_g2o'
    filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
    factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
    marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

    A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

    sync_convergence_threshold = 1*10**-0 # convergence threshold
    convergence_type = 'all' #all
    simulation_convergence_threshold = 1*10**-10

    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                        b, 
                                                                                                        max_iter=1_000, 
                                                                                                        mae=True if convergence_type == 'mae' else False, 
                                                                                                        convergence_threshold=simulation_convergence_threshold,
                                                                                                        show=True)
    final_mean_sync = list(mu_i)

    if iteration < 1_000:
        break
    else:
        iteration = float("inf")
        print("=========== RESTART ===========")

iteration: 1
0.025196008670395724
-----
iteration: 2
0.0006189516168088241
-----
iteration: 3
5.844281343917183e-05
-----
iteration: 4
8.723896685171526e-06
-----
iteration: 5
1.0043103865814744e-06
-----
iteration: 6
1.4670289153417872e-07
-----
iteration: 7
1.1180720481595927e-08
-----
iteration: 8
7.362751430292566e-10
-----
iteration: 9
0.0
-----


In [3]:
print(A.shape[0])

1228


### Hardware

In [4]:
ASYNC_ITER = 10
NODE_UPDT_PE = 100
PEs = 1

async_convergence_threshold = 1*10**-5

In [5]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=False, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.016574562178015374
-----
iteration: 2
0.016574562178015374
-----
iteration: 3
0.016574562178015374
-----
iteration: 4
0.016574562178015374
-----
iteration: 5
0.016574562178015374
-----
iteration: 6
0.016574562178015374
-----
iteration: 7
0.016574562178015374
-----
iteration: 8
0.016574562178015374
-----
iteration: 9
0.01321581120174374
-----
iteration: 10
0.01321581120174374
-----
iteration: 11
0.01321581120174374
-----
iteration: 12
0.01321581120174374
-----
iteration: 13
0.01321581120174374
-----
iteration: 14
0.01321581120174374
-----
iteration: 15
0.01321581120174374
-----
iteration: 16
0.01321581120174374
-----
iteration: 17
0.01321581120174374
-----
iteration: 18
0.01321581120174374
-----
iteration: 19
0.01321581120174374
-----
iteration: 20
0.01321581120174374
-----
iteration: 21
0.01321581120174374
-----
iteration: 22
0.01321581120174374
-----
iteration: 23
0.01321581120174374
-----
iteration: 24
0.01321581120174374
-----
iteration: 25
0.011794996382577572
-----
